In [ ]:
#!pip install -q kaggle

In [ ]:
'''from google.colab import files
files.upload()'''

In [ ]:
'''!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!rm kaggle.json
!kaggle competitions download -c tuberculosis-drug-resistance-prediction-emb'''

  0% 0.00/5.43k [00:00<?, ?B/s]
100% 5.43k/5.43k [00:00<00:00, 9.11MB/s]
  0% 0.00/74.1k [00:00<?, ?B/s]
100% 74.1k/74.1k [00:00<00:00, 69.1MB/s]
  0% 0.00/91.0k [00:00<?, ?B/s]
100% 91.0k/91.0k [00:00<00:00, 28.9MB/s]
  0% 0.00/443k [00:00<?, ?B/s]
100% 443k/443k [00:00<00:00, 61.8MB/s]


In [ ]:
'''!unzip ./X_trainData_1.csv.zip
!rm ./X_trainData_1.csv.zip'''

Archive:  ./X_trainData_1.csv.zip
  inflating: X_trainData_1.csv       


In [ ]:
import pandas as pd
df = pd.read_csv('X_trainData_1.csv')
#df

In [ ]:
df1 = pd.read_csv('Y_trainData_1.csv')
#df1

In [ ]:
for i in df1.columns:
  print(df1[i].value_counts())

 1    2057
 0    1278
-1      58
Name: RIF, dtype: int64
 1    1832
 0    1524
-1      37
Name: INH, dtype: int64
 1    2246
 0     695
-1     452
Name: PZA, dtype: int64
 1    2346
 0     973
-1      74
Name: EMB, dtype: int64
-1    1312
 1    1061
 0    1020
Name: STR, dtype: int64
-1    2214
 1     942
 0     237
Name: CIP, dtype: int64
-1    2054
 1     752
 0     587
Name: CAP, dtype: int64
-1    2033
 1    1127
 0     233
Name: AMK, dtype: int64
-1    2056
 1    1070
 0     267
Name: MOXI, dtype: int64
-1    2703
 1     603
 0      87
Name: OFLX, dtype: int64
-1    2110
 1    1013
 0     270
Name: KAN, dtype: int64


In [ ]:
df = pd.read_csv('X_trainData_1.csv')
df1 = pd.read_csv('Y_trainData_1.csv')

df.drop('SNP_CN_2714366_C967A_V323L_eis',axis=1,inplace=True)
df.drop('SNP_I_2713795_C329T_inter_Rv2415c_eis',axis=1,inplace=True)
df.drop('SNP_I_2713872_C252A_inter_Rv2415c_eis',axis=1,inplace=True)
count=0
for ind in df1.index:
  x = df1['EMB'][ind]
  if x == -1:
    count+=1
    df.drop(index=ind,inplace=True)
    df1.drop(index=ind,inplace=True)
print(count)
df['EMB'] = df1['EMB']



74


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedShuffleSplit,RandomizedSearchCV,GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB
import numpy as np
import pickle

In [ ]:
ss = StratifiedShuffleSplit(n_splits=1,test_size=0.1,random_state=42)

traindata1 = df.drop(columns=['EMB'],axis=1)
y1 = df['EMB']

for train_index, test_index in ss.split(traindata1,y1):
  #print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = traindata1.iloc[train_index], traindata1.iloc[test_index]
  y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]


X_train.shape,X_test.shape

((2987, 219), (332, 219))

In [ ]:
y_train.value_counts()

1    2111
0     876
Name: EMB, dtype: int64

In [ ]:
## Hyper Parameter Optimization

params={
 "learning_rate"    : [0.01,0.03,0.05, 0.10,0.12, 0.15,0.18, 0.20, 0.25, 0.30,0.35,0.4] ,
 "max_depth"        : [ 6,8, 10, 12,14, 15,16],
 "gamma"            : [ 0.0,0.2,0.4 ],
 'n_estimators'     : [50,60,70,80,85,90,100,110,120,150,170,190,200,250,300,350,370,400]   
}
params1 = {
    'C' :[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,1,1,2,1,3,1,5,1.6,1.7,1.9,2,2.2,2.4,2.6,2.8,3,3.2,3.4],
   'kernel' : ['rbf','linear'],
   'degree':[1,2,3,4,5,6,7,8],
    'class_weight':['balanced',''],
  'decision_function_shape':['ovo', 'ovr']
}
params2 = {
    'C' :[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,1,1,2,1,3,1,5,1.6,1.7,1.9,2,2.2],
    'class_weight':['balanced',''],
    'solver':['newton-cg','lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter':[20,40,60,80,100,120,140,180,200,230,250,270,300]
}
params3 = {
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12,14, 15,16,18],
 "min_samples_split" : [ 1,2, 3,4, 5,6, 7 ],
 "max_features" : [ 'auto','sqrt','log2' ],
 'n_estimators'     : [50,60,70,80,85,90,100,110,120,150,170,190,200,250,300,350]   
}

In [ ]:
classifier= RandomForestClassifier(random_state=0)
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=25,scoring='roc_auc'
                                 ,n_jobs=-1,cv=2,verbose=3)

grid_search=GridSearchCV(classifier,param_grid=params3,scoring='roc_auc',
                                 n_jobs=-1,cv=2,verbose=3)

In [ ]:
grid_search.fit(X_train,y_train)

Fitting 2 folds for each of 3696 candidates, totalling 7392 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 508 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 1148 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 1564 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 2044 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 2588 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 3868 tasks      | elapsed: 16.0min
[Parallel(n_jobs=-1)]: Done 4604 tasks      | elapsed: 19.4min
[Parallel(n_jobs=-1)]: Done 5404 tasks      | elapsed: 23.2min
[Parallel(n_jobs=-1)]: Done 6268 tasks      | elapsed: 27.7min
[Parallel(n_jobs=-1)]: Done 7196 tasks      | e

GridSearchCV(cv=2, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=0,
                                   

In [ ]:
print(grid_search.best_estimator_)
grid_search.best_score_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=350,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)


0.9649013691580479

In [ ]:
lr = LogisticRegression(C=1, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=20, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))

y_pred = lr.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9216867469879518
[[ 92   5]
 [ 21 214]]
              precision    recall  f1-score   support

           0       0.81      0.95      0.88        97
           1       0.98      0.91      0.94       235

    accuracy                           0.92       332
   macro avg       0.90      0.93      0.91       332
weighted avg       0.93      0.92      0.92       332



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [ ]:
svm = SVC(C=2, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovo', degree=1, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)


svm.fit(X_train, y_train)
print(svm.score(X_test, y_test))

y_pred = svm.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9186746987951807
[[ 89   8]
 [ 19 216]]
              precision    recall  f1-score   support

           0       0.82      0.92      0.87        97
           1       0.96      0.92      0.94       235

    accuracy                           0.92       332
   macro avg       0.89      0.92      0.90       332
weighted avg       0.92      0.92      0.92       332



In [ ]:
rf = RandomForestClassifier(max_depth=17, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))

y_pred = rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9216867469879518
[[ 90   7]
 [ 19 216]]
              precision    recall  f1-score   support

           0       0.83      0.93      0.87        97
           1       0.97      0.92      0.94       235

    accuracy                           0.92       332
   macro avg       0.90      0.92      0.91       332
weighted avg       0.93      0.92      0.92       332



In [ ]:
xf = XGBClassifier(
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              n_estimators=130)

xf.fit(X_train, y_train)

print(xf.score(X_train, y_train))
print(xf.score(X_test, y_test))
y_pred = xf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9762303314362236
0.9216867469879518
[[ 89   8]
 [ 18 217]]
              precision    recall  f1-score   support

           0       0.83      0.92      0.87        97
           1       0.96      0.92      0.94       235

    accuracy                           0.92       332
   macro avg       0.90      0.92      0.91       332
weighted avg       0.93      0.92      0.92       332



In [ ]:

score=cross_val_score(xf,traindata1,y1,cv=2,scoring='roc_auc')
print(score)
score.mean()

[0.92562814 0.99240455]


0.9590163435468643

In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.utils import class_weight
from keras import optimizers
tf.__version__


'2.3.0'

In [ ]:
X_train.shape

(2987, 219)

In [ ]:
!rm -r ./model/
!rm ./model.zip

In [ ]:
model = tf.keras.models.Sequential([
tf.keras.layers.InputLayer(input_shape=219),
tf.keras.layers.Dense(128,activation='relu'),
#tf.keras.layers.Dropout(0.05),
tf.keras.layers.Dense(16,activation='relu'),
#tf.keras.layers.Dense(8,activation='relu'),
tf.keras.layers.Dense(4,activation='relu'),
tf.keras.layers.Dense(1,activation='sigmoid'),
])


In [ ]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(),
              )
model.summary()
weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
print(weights)
temp = {0:weights[0],1:weights[1]}
type(temp)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               28160     
_________________________________________________________________
dense_5 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 68        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 5         
Total params: 30,297
Trainable params: 30,297
Non-trainable params: 0
_________________________________________________________________
[1.70490868 0.7074846 ]


dict

In [ ]:
#!unzip model_EMB.zip
#!rm model_EMB.zip
model = tf.keras.models.load_model('./model_EMB/')
model.summary()

Archive:  model_EMB.zip
replace model/saved_model.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: model/saved_model.pb    
  inflating: model/variables/variables.index  
  inflating: model/variables/variables.data-00000-of-00001  
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               28160     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 68        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 30,297
Trainable params: 30,297
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train,y_train,epochs=1,validation_data=(X_test,y_test),class_weight=temp)

94/94 [==============================] - 0s 3ms/step - loss: 0.1521 - val_loss: 0.2111


In [ ]:
model.save('model_EMB')
!zip -r model_EMB.zip ./model_EMB

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model/assets
  adding: model/ (stored 0%)
  adding: model/assets/ (stored 0%)
  adding: model/saved_model.pb (deflated 89%)
  adding: model/variables/ (stored 0%)
  adding: model/variables/variables.index (deflated 63%)
  adding: model/variables/variables.data-00000-of-00001 (deflated 9%)


In [ ]:
preds = model.predict(X_test)
preds1 = np.where(preds>0.5,1,0)
preds2 = np.where(model.predict(X_train)>0.5,1,0)
print(np.unique(preds1,return_counts=True))
print(accuracy_score(y_train,preds2))
print(accuracy_score(y_test,preds1))
print(confusion_matrix(y_test,preds1))
print(roc_auc_score(y_test,preds))

(array([0, 1]), array([100, 232]))
0.9561432875795112
0.9126506024096386
[[ 84  13]
 [ 16 219]]
0.9666593551217372


In [ ]:
X_testdata = pd.read_csv('X_testData_1.csv')
Y_testdata_nolabel = pd.read_csv('Y_testData_1_nolabels_EMB.csv')
output_id = set(Y_testdata_nolabel['ID'])
len(output_id)

943

In [ ]:
X_testdata.drop('SNP_CN_2714366_C967A_V323L_eis',axis=1,inplace=True)
X_testdata.drop('SNP_I_2713795_C329T_inter_Rv2415c_eis',axis=1,inplace=True)
X_testdata.drop('SNP_I_2713872_C252A_inter_Rv2415c_eis',axis=1,inplace=True)

generate_id = X_testdata['ID']
X_testdata.drop('ID',axis=1,inplace=True)
#generate_id

In [ ]:
#Run this to produce output for ML models
output_pred = lr.predict_proba(X_testdata)
out = pd.DataFrame({'EMB':output_pred[:,1]})
out['ID'] = generate_id
out = out[['ID','EMB']]
#out
for ind in out.index:
    if out['ID'][ind] not in output_id:
        out.drop(ind,inplace=True)
print(out['EMB'].value_counts())
sorted(out['EMB'])[:10]

0.986920    48
0.989678    30
0.998692    24
0.990009    18
0.940651    17
            ..
0.984181     1
0.978011     1
0.086890     1
0.174514     1
0.993396     1
Name: EMB, Length: 659, dtype: int64


[0.005419226584863833,
 0.005419226584863833,
 0.007488471306583289,
 0.0096623970446826,
 0.012786904287442374,
 0.01327887353641731,
 0.014706827808841654,
 0.014706827808841654,
 0.015721299343458606,
 0.01743457847357621]

In [ ]:
#Run this to produce output for neural network
output_pred = model.predict(X_testdata)
out = pd.DataFrame({'EMB':output_pred[:,0]})
out['ID'] = generate_id
out = out[['ID','EMB']]
#out
for ind in out.index:
    if out['ID'][ind] not in output_id:
        out.drop(ind,inplace=True)
print(out['EMB'].value_counts())
#sorted(out['EMB'])[:10]

In [ ]:
out.to_csv('Group_7_output.csv',index=None)